# Experiment 1

In [1]:
using KLIEPInference
using ProximalBase, CoordinateDescent
using LinearAlgebra, SparseArrays, Statistics, Random
using Distributions, JLD, PyPlot

In [2]:
function experiment1(file, idx, nx, ny, nrep)
    θx = pack(read(file, "Θx"))
    θy = pack(read(file, "Θy"))
    close(file)
    
    p = length(θx)
    
    # for both steps, we use autoscaling procedures with canonical choices of λ
    λ1 = 1.01 * quantile(Normal(), 1. - 0.05 / p)
    λ2 = sqrt(2. * log(p) / ny)
    
    b = zeros(Float64, 4, nrep)
    z = zeros(Float64, 4, nrep)
    
    for rep = 1:nrep
        if mod(rep, div(nrep, 10))==0
            println("$(rep) / $(nrep)")
        end
        
        # generate samples
        spl = IsingSampler(θx; thin=2000)
        X = rand(spl, nx)
        spl = IsingSampler(θy; thin=2000)
        Y = rand(spl, ny)
        
        Ψx = Ψising(X)
        Ψy = Ψising(Y)
        
        # oracle
        supp = findall(!iszero, θx-θy)
        θ = spKLIEP_refit!(SparseIterate(p), Ψx, Ψy, supp)
        
        # oracle estimate
        H = KLIEP_Hessian(θ, Ψy)
        ω = Hinv_row_refit!(SparseIterate(p), H, idx, supp)
        σ = stderr_KLIEP(θ, ω, Ψx, Ψy)
        
        b[1,rep] = θ[idx] - (θx[idx] - θy[idx])
        z[1,rep] = b[1,rep] / σ
        
        # step 1
        θ = spKLIEP(Ψx, Ψy, λ1, CD_KLIEP(); loadings=true)
        supp = sort(union(idx, findall(!iszero, θ)))
        spKLIEP_refit!(θ, Ψx, Ψy, supp)
        
        # naïve re-fitted estimate
        H = KLIEP_Hessian(θ, Ψy)
        ω = Hinv_row_refit!(SparseIterate(p), H, idx, supp)
        σ = stderr_KLIEP(θ, ω, Ψx, Ψy)
        
        b[2,rep] = θ[idx] - (θx[idx] - θy[idx])
        z[2,rep] = b[2,rep] / σ
        
        # step 2
        H = KLIEP_Hessian(θ, Ψy)
        ω = Hinv_row(H, idx, λ2)
        supp = sort(union(idx, findall(!iszero, ω)))
        Hinv_row_refit!(ω, H, idx, supp)
        
        # SparKLIE+1 and +2
        σ1 = stderr_KLIEP(θ, ω, Ψx, Ψy)
        θ1, θ2 = debias_KLIEP(idx, θ, ω, Ψx, Ψy)
        σ2 = stderr_KLIEP(θ, ω, Ψx, Ψy)
        
        b[3,rep], b[4,rep] = θ1 - (θx[idx] - θy[idx]), θ2 - (θx[idx] - θy[idx])
        z[3,rep], z[4,rep] = b[3,rep] / σ1, b[4,rep] / σ2
    end
    
    return b, z
end

experiment1 (generic function with 1 method)

In [3]:
function qqplot(z; color="black")
    n = length(z)

    grid = [(1 / (n + 1)):(1 / (n + 1)):(1.0 - (1 / (n + 1)));]

    qz = quantile(z, grid)
    qd = quantile.(Ref(Distributions.Normal()), grid)

    lims = 3.290
    x = range(-lims, stop=lims)

    plot(x, x, color="grey", linestyle=":", linewidth=.25)
    scatter(qz, qd, s=.75, color=color)

    xlim([-lims, lims])
    ylim([-lims, lims])

    nothing
end

qqplot (generic function with 1 method)

In [4]:
Random.seed!(1218)

MersenneTwister(UInt32[0x000004c2], Random.DSFMT.DSFMT_state(Int32[-854650020, 1073556053, 1620855344, 1073630175, 1698263943, 1073736988, 691512570, 1073738378, -673392900, 1073121727  …  47415821, 1072887989, -496483395, 1073429484, -1104767902, 678491891, -1379894893, 290756459, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x0000000000000

## $m = 25$, $n_x = 150$, $n_y = 300$

### Chain (1)

In [5]:
file = jldopen("/Users/byolkim/Documents/KLIEPInference.jl/exper/exper1/graphs/chain1_25.jld", "r")
idx = KLIEPInference.trimap(5,6)  #edge of interest
b, z = experiment1(file, idx, 150, 300, 1000);

100 / 1000
200 / 1000
300 / 1000
400 / 1000
500 / 1000
600 / 1000
700 / 1000
800 / 1000
900 / 1000
1000 / 1000


In [6]:
println("avg bias: $(mean(b, dims=2))")
println("coverage: $(mean(abs.(z) .< quantile(Normal(), 0.975), dims=2))")

avg bias: [-0.002411635399094899; 0.08423432452518499; -0.014900303256989855; -0.0027481200337800917]
coverage: [0.968; 0.863; 0.953; 0.966]


In [7]:
x = range(-3.290, stop=3.290, length=100)
y = pdf.(Normal(), x)

fig = figure(figsize=(5.125,3.844), dpi=300)

ax = subplot(2, 3, 1)
qqplot(z[1,:], color="grey")
qqplot(z[2,:])
title("naive re-fitting", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 2)
qqplot(z[1,:], color="grey")
qqplot(z[3,:])
title("SparKLIE+1", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 3)
qqplot(z[1,:], color="grey")
qqplot(z[4,:])
title("SparKLIE+2", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 4)
plt[:hist](z[2,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 5)
plt[:hist](z[3,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 6)
plt[:hist](z[4,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)
tight_layout()

show(fig)

savefig("/Users/byolkim/Documents/KLIEP_inference/draft/revision_v3/exper1_chain1_25.png")
close(fig)

Figure(PyObject <Figure size 1537.5x1153.2 with 6 Axes>)

### Chain (2)

In [8]:
file = jldopen("/Users/byolkim/Documents/KLIEPInference.jl/exper/exper1/graphs/chain2_25.jld", "r")
idx = KLIEPInference.trimap(5,6)  #edge of interest
b, z = experiment1(file, idx, 150, 300, 1000);

100 / 1000
200 / 1000
300 / 1000
400 / 1000
500 / 1000
600 / 1000
700 / 1000
800 / 1000
900 / 1000
1000 / 1000


In [9]:
println("avg bias: $(mean(b, dims=2))")
println("coverage: $(mean(abs.(z) .< quantile(Normal(), 0.975), dims=2))")

avg bias: [-0.009234481804009794; 0.06951258323141063; 0.0047431406699008334; -0.009578163439097166]
coverage: [0.951; 0.888; 0.941; 0.943]


In [10]:
x = range(-3.290, stop=3.290, length=100)
y = pdf.(Normal(), x)

fig = figure(figsize=(5.125,3.844), dpi=300)

ax = subplot(2, 3, 1)
qqplot(z[1,:], color="grey")
qqplot(z[2,:])
title("naive re-fitting", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 2)
qqplot(z[1,:], color="grey")
qqplot(z[3,:])
title("SparKLIE+1", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 3)
qqplot(z[1,:], color="grey")
qqplot(z[4,:])
title("SparKLIE+2", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 4)
plt[:hist](z[2,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 5)
plt[:hist](z[3,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 6)
plt[:hist](z[4,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)
tight_layout()

show(fig)

savefig("/Users/byolkim/Documents/KLIEP_inference/draft/revision_v3/exper1_chain2_25.png")
close(fig)

Figure(PyObject <Figure size 1537.5x1153.2 with 6 Axes>)

### Tree (1)

In [11]:
file = jldopen("/Users/byolkim/Documents/KLIEPInference.jl/exper/exper1/graphs/tree1_25.jld", "r")
idx = KLIEPInference.trimap(1,3)  #edge of interest
b, z = experiment1(file, idx, 150, 300, 1000);

100 / 1000
200 / 1000
300 / 1000
400 / 1000
500 / 1000
600 / 1000
700 / 1000
800 / 1000
900 / 1000
1000 / 1000


In [12]:
println("avg bias: $(mean(b, dims=2))")
println("coverage: $(mean(abs.(z) .< quantile(Normal(), 0.975), dims=2))")

avg bias: [-0.006553689802310843; 0.07077168463067289; -0.009544536979063517; -0.028343262290836863]
coverage: [0.973; 0.925; 0.943; 0.98]


In [13]:
x = range(-3.290, stop=3.290, length=100)
y = pdf.(Normal(), x)

fig = figure(figsize=(5.125,3.844), dpi=300)

ax = subplot(2, 3, 1)
qqplot(z[1,:], color="grey")
qqplot(z[2,:])
title("naive re-fitting", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 2)
qqplot(z[1,:], color="grey")
qqplot(z[3,:])
title("SparKLIE+1", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 3)
qqplot(z[1,:], color="grey")
qqplot(z[4,:])
title("SparKLIE+2", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 4)
plt[:hist](z[2,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 5)
plt[:hist](z[3,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 6)
plt[:hist](z[4,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)
tight_layout()

show(fig)

savefig("/Users/byolkim/Documents/KLIEP_inference/draft/revision_v3/exper1_tree1_25.png")
close(fig)

Figure(PyObject <Figure size 1537.5x1153.2 with 6 Axes>)

### Tree (2)

In [14]:
file = jldopen("/Users/byolkim/Documents/KLIEPInference.jl/exper/exper1/graphs/tree2_25.jld", "r")
idx = KLIEPInference.trimap(1,3)  #edge of interest
b, z = experiment1(file, idx, 150, 300, 1000);

100 / 1000
200 / 1000
300 / 1000
400 / 1000
500 / 1000
600 / 1000
700 / 1000
800 / 1000
900 / 1000
1000 / 1000


In [15]:
println("avg bias: $(mean(b, dims=2))")
println("coverage: $(mean(abs.(z) .< quantile(Normal(), 0.975), dims=2))")

avg bias: [-0.012516278732856576; 0.04378068971477958; 0.012023583129567899; -0.007966158067718132]
coverage: [0.974; 0.96; 0.954; 0.99]


In [16]:
x = range(-3.290, stop=3.290, length=100)
y = pdf.(Normal(), x)

fig = figure(figsize=(5.125,3.844), dpi=300)

ax = subplot(2, 3, 1)
qqplot(z[1,:], color="grey")
qqplot(z[2,:])
title("naive re-fitting", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 2)
qqplot(z[1,:], color="grey")
qqplot(z[3,:])
title("SparKLIE+1", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 3)
qqplot(z[1,:], color="grey")
qqplot(z[4,:])
title("SparKLIE+2", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 4)
plt[:hist](z[2,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 5)
plt[:hist](z[3,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 6)
plt[:hist](z[4,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)
tight_layout()

show(fig)

savefig("/Users/byolkim/Documents/KLIEP_inference/draft/revision_v3/exper1_tree2_25.png")
close(fig)

Figure(PyObject <Figure size 1537.5x1153.2 with 6 Axes>)

## $m = 50$, $n_x = 300$, $n_y = 600$

### Chain (1)

In [17]:
file = jldopen("/Users/byolkim/Documents/KLIEPInference.jl/exper/exper1/graphs/chain1_50.jld", "r")
idx = KLIEPInference.trimap(5,6)  #edge of interest
b, z = experiment1(file, idx, 300, 600, 1000);

100 / 1000
200 / 1000
300 / 1000
400 / 1000
500 / 1000
600 / 1000
700 / 1000
800 / 1000
900 / 1000
1000 / 1000


In [18]:
println("avg bias: $(mean(b, dims=2))")
println("coverage: $(mean(abs.(z) .< quantile(Normal(), 0.975), dims=2))")

avg bias: [-0.001598659715388254; 0.07824713319031354; -0.022243522812742313; -0.015141925312805234]
coverage: [0.961; 0.823; 0.957; 0.969]


In [19]:
x = range(-3.290, stop=3.290, length=100)
y = pdf.(Normal(), x)

fig = figure(figsize=(5.125,3.844), dpi=300)

ax = subplot(2, 3, 1)
qqplot(z[1,:], color="grey")
qqplot(z[2,:])
title("naive re-fitting", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 2)
qqplot(z[1,:], color="grey")
qqplot(z[3,:])
title("SparKLIE+1", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 3)
qqplot(z[1,:], color="grey")
qqplot(z[4,:])
title("SparKLIE+2", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 4)
plt[:hist](z[2,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 5)
plt[:hist](z[3,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 6)
plt[:hist](z[4,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)
tight_layout()

show(fig)

savefig("/Users/byolkim/Documents/KLIEP_inference/draft/revision_v3/exper1_chain1_50.png")
close(fig)

Figure(PyObject <Figure size 1537.5x1153.2 with 6 Axes>)

### Chain (2)

In [20]:
file = jldopen("/Users/byolkim/Documents/KLIEPInference.jl/exper/exper1/graphs/chain2_50.jld", "r")
idx = KLIEPInference.trimap(5,6)  #edge of interest
b, z = experiment1(file, idx, 300, 600, 1000);

100 / 1000
200 / 1000
300 / 1000
400 / 1000
500 / 1000
600 / 1000
700 / 1000
800 / 1000
900 / 1000
1000 / 1000


In [21]:
println("avg bias: $(mean(b, dims=2))")
println("coverage: $(mean(abs.(z) .< quantile(Normal(), 0.975), dims=2))")

avg bias: [0.0019267987094414311; 0.08070795003280348; 0.01855262611218014; -0.0008054816816843565]
coverage: [0.958; 0.817; 0.949; 0.953]


In [22]:
x = range(-3.290, stop=3.290, length=100)
y = pdf.(Normal(), x)

fig = figure(figsize=(5.125,3.844), dpi=300)

ax = subplot(2, 3, 1)
qqplot(z[1,:], color="grey")
qqplot(z[2,:])
title("naive re-fitting", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 2)
qqplot(z[1,:], color="grey")
qqplot(z[3,:])
title("SparKLIE+1", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 3)
qqplot(z[1,:], color="grey")
qqplot(z[4,:])
title("SparKLIE+2", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 4)
plt[:hist](z[2,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 5)
plt[:hist](z[3,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 6)
plt[:hist](z[4,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)
tight_layout()

show(fig)

savefig("/Users/byolkim/Documents/KLIEP_inference/draft/revision_v3/exper1_chain2_50.png")
close(fig)

Figure(PyObject <Figure size 1537.5x1153.2 with 6 Axes>)

### Tree (1)

In [23]:
file = jldopen("/Users/byolkim/Documents/KLIEPInference.jl/exper/exper1/graphs/tree1_50.jld", "r")
idx = KLIEPInference.trimap(1,3)  #edge of interest
b, z = experiment1(file, idx, 300, 600, 1000);

100 / 1000
200 / 1000
300 / 1000
400 / 1000
500 / 1000
600 / 1000
700 / 1000
800 / 1000
900 / 1000
1000 / 1000


In [24]:
println("avg bias: $(mean(b, dims=2))")
println("coverage: $(mean(abs.(z) .< quantile(Normal(), 0.975), dims=2))")

avg bias: [-0.010455539919384027; 0.0779980265485224; -0.010693215292402055; -0.020413045983369616]
coverage: [0.972; 0.913; 0.951; 0.978]


In [25]:
x = range(-3.290, stop=3.290, length=100)
y = pdf.(Normal(), x)

fig = figure(figsize=(5.125,3.844), dpi=300)

ax = subplot(2, 3, 1)
qqplot(z[1,:], color="grey")
qqplot(z[2,:])
title("naive re-fitting", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 2)
qqplot(z[1,:], color="grey")
qqplot(z[3,:])
title("SparKLIE+1", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 3)
qqplot(z[1,:], color="grey")
qqplot(z[4,:])
title("SparKLIE+2", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 4)
plt[:hist](z[2,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 5)
plt[:hist](z[3,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 6)
plt[:hist](z[4,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)
tight_layout()

show(fig)

savefig("/Users/byolkim/Documents/KLIEP_inference/draft/revision_v3/exper1_tree1_50.png")
close(fig)

Figure(PyObject <Figure size 1537.5x1153.2 with 6 Axes>)

### Tree (2)

In [26]:
file = jldopen("/Users/byolkim/Documents/KLIEPInference.jl/exper/exper1/graphs/tree2_50.jld", "r")
idx = KLIEPInference.trimap(1,3)  #edge of interest
b, z = experiment1(file, idx, 300, 600, 1000);

100 / 1000
200 / 1000
300 / 1000
400 / 1000
500 / 1000
600 / 1000
700 / 1000
800 / 1000
900 / 1000
1000 / 1000


In [27]:
println("avg bias: $(mean(b, dims=2))")
println("coverage: $(mean(abs.(z) .< quantile(Normal(), 0.975), dims=2))")

avg bias: [-0.0018140440077490188; 0.05426107433175982; 0.003722867497560637; -0.0038123635865270123]
coverage: [0.968; 0.926; 0.949; 0.99]


In [28]:
x = range(-3.290, stop=3.290, length=100)
y = pdf.(Normal(), x)

fig = figure(figsize=(5.125,3.844), dpi=300)

ax = subplot(2, 3, 1)
qqplot(z[1,:], color="grey")
qqplot(z[2,:])
title("naive re-fitting", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 2)
qqplot(z[1,:], color="grey")
qqplot(z[3,:])
title("SparKLIE+1", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 3)
qqplot(z[1,:], color="grey")
qqplot(z[4,:])
title("SparKLIE+2", size="xx-small")
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 4)
plt[:hist](z[2,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 5)
plt[:hist](z[3,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)

ax = subplot(2, 3, 6)
plt[:hist](z[4,:], 100, density=true)
plot(x,y)
xlim(-3.290,3.290)
ax[:tick_params]("both", labelsize="xx-small", length=2, pad=2)
tight_layout()

show(fig)

savefig("/Users/byolkim/Documents/KLIEP_inference/draft/revision_v3/exper1_tree2_50.png")
close(fig)

Figure(PyObject <Figure size 1537.5x1153.2 with 6 Axes>)